<a href="https://colab.research.google.com/github/plaban1981/Agents/blob/main/LLM_Router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qU langchain
!pip install -qU groq
!pip install -qU langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 5.5 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [5]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [12]:
def openrouter_completion(model_name, user_query, system_message=None):
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": user_query})
    completion = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    return completion.choices[0].message.content

In [14]:
def router(user_input):
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": """You are a helpful router designed to output JSON. you decide which route user input belongs to based on the type of the user query
             return your response as follows:
             {{
             "route": "route_name", can be one of the following:
             - "llama-3.1-70b-versatile"- for more complex, in depth questions and queries except code
             - "llama3-70b-8192" for all code queries.
             - "gemma-7b-it" for other simple daily conversation like regular speak
             }}"""},
            {"role": "user", "content": user_input}
        ]
    )
    print(type(response.choices[0].message.content))
    return response.choices[0].message.content

In [15]:
import json
while True:
    user_input = input("Please enter your question (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    # Attempt to parse JSON, handle potential errors
    try:
        response_text = router(user_input)
        model = json.loads(response_text)
        model = model["route"]
    except json.JSONDecodeError:
        print("Error: Invalid JSON returned from router. Please review the router's output.")
        print(f"Router raw output: {response_text}") # Print raw output for debugging
        continue # Skip to next iteration of the loop

    if model == "llama3-8b-8192":
        system_message = "You are a masterful english literary"
        print(openrouter_completion(model, user_input, system_message))
    elif model == "llama-3.1-70b-versatile":
        print(openrouter_completion(model, user_input))
    elif model == "gemma-7b-it":
        print(openrouter_completion(model, user_input))

Please enter your question (or type 'exit' to quit): What is the Contribution of Shakesphere to English Literature ?
<class 'str'>
I think you meant "Shakespeare"!

William Shakespeare (1564-1616) is widely regarded as one of the greatest writers in the English language and the world's pre-eminent dramatist. His contributions to English literature are immense and far-reaching. Here are some of the most significant:

1. **Invention of New Words and Phrases**: Shakespeare introduced many words and phrases that are still in use today. It's estimated that he contributed over 1700 words to the English language, including "assassination," "bedazzled," "fashionable," and "lonely."
2. **Development of the English Sonnet**: Shakespeare popularized the sonnet, a 14-line poem with a specific rhyme scheme. His sonnets are still studied and admired for their beauty, complexity, and emotional depth.
3. **Shaping the English Language**: Shakespeare's writing helped to standardize English spelling and